 **1. Сравните метрики hit_rate@k, precision@k.   Какую метрику использовать предпочтительно и почему?  Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?**

Метрика hitrate@k показывает, было ли хоть одно попадание в списке К рекомендованных товаров. 
Вероятно, информативно использовать эту метрику в случаях, когда мы рекомендуем однотипные товары, из которых обычно клиент выбирает какой-то один. Например, Geekbrains может порекомендовать несколько курсов. Если рекомендация сработала, то клиент выберет и купит один курс из списка. 
Также можно порекомендовать купить машину, ее тоже либо купят одну из списка, либо не купят.

В случае, если нам нужно, чтобы клиент купил или оценил несколько товаров из списка, предпочтительно использовать precision@k. Например, рекомендации продуктов к покупке, фильмов к просмотру, статей.




**2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в логарифм.**


Логарифм- плавно возрастающая функция, поместив логарифм в знаменатель мы плавно снижаем значимость каждой последующей рекомендации. При этом разница в значимости первых рекомендаций больше, чем разница рекомендаций в конце.
Вместо логарифма можно использовать другие плавно возрастающие функции, например, sqrt(x).


**3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).**

**4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)
IDCG должен считаться для всех 8 позиций рекомендаций**

**pr@8**

In [17]:
import numpy as np

recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
bought = [1, 3, 5, 7, 9, 11]

$pr@8 = \frac{0 + 1 + 1 +0 + 1 + 1 + 0 + 0}{1+1+1+1+1+1+1+1} = 50\%$  

In [16]:
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [20]:
precision_at_k(recommended, bought, 5)

0.5

**rec@8**

$rec@8 = \frac{0 + 0 + 1 +1 + 1 + 1}{1+1+1+1+1+1} = 66,6\%$  

In [31]:
def recall_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [32]:
recall_at_k(recommended, bought, 8)

0.6666666666666666

**AP@8**

$pr@8 = \frac{0 + 1 + 1 +0 + 1 + 1 + 0 + 0}{1+1+1+1+1+1+1+1}\$

$ap@8 = \(pr@2 + pr@3 + pr@5 + pr@6) / 8 = (1/2 + 2/3 + 3/5 + 4/6)*1/8 = 0,304\$

In [132]:
def ap_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / k 
    
    return result

In [133]:
ap_k(recommended, bought, 8)

0.30416666666666664

**NDCG@8**

$NDCG = \frac{DCG}{ideal DCG}$

In [125]:
dcg = (0+1/np.log(2)+1/np.log(3)+1/np.log(5)+1/np.log(6))/8

In [126]:
ideal_dcg = (1+1/np.log(2)+1/np.log(3)+1/np.log(4)+1/np.log(5)+1/np.log(6)+1/np.log(7)+1/np.log(8))/8

In [127]:
dcg/ideal_dcg

0.5653142737255068

In [130]:
def ndcg_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    sum_ += flags[0]/1
    for i in range(1, k): 
        if flags[i] == True:
            d = 1/np.log(i+1)
            sum_ += d
            
    dcg = sum_ / k
    
    ideal = 1
    for i in range(1, k): 
        ideal += 1/np.log(i+1)
        
    ideal_dcg = ideal/k
    
    result = dcg/ideal_dcg
    
    return result

In [131]:
ndcg_at_k(recommended, bought, 8)

0.5653142737255068

**MRR@8**

In [140]:
(1/2+1/3+1/5+1/6)/8

0.15

In [141]:
def reciprocal_rank(recommended_list, bought_list,k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [143]:
reciprocal_rank(recommended, bought, 8)

0.15